In [431]:
import pandas as pd
import numpy as np

ruta = "Calamar.xlsx"  
df = pd.read_excel(ruta)
df.head

<bound method NDFrame.head of        CodigoEstacion                NombreEstacion     Latitud      Longitud  \
0            29037020            CALAMAR [29037020]  1025386111 -7.491169e+09   
1            29037020            CALAMAR [29037020]  1025386111 -7.491169e+09   
2            29037020            CALAMAR [29037020]  1025386111 -7.491169e+09   
3            29037020            CALAMAR [29037020]  1025386111 -7.491169e+09   
4            29037020            CALAMAR [29037020]  1025386111 -7.491169e+09   
...               ...                           ...         ...           ...   
27736        14015060  ISLAS DEL ROSARIO [14015060]  1018333333 -7.575000e+01   
27737        14015060  ISLAS DEL ROSARIO [14015060]  1018333333 -7.575000e+01   
27738        14015060  ISLAS DEL ROSARIO [14015060]  1018333333 -7.575000e+01   
27739        14015060  ISLAS DEL ROSARIO [14015060]  1018333333 -7.575000e+01   
27740        14015060  ISLAS DEL ROSARIO [14015060]  1018333333 -7.575000e+01  

In [432]:
# Seleccionar las columnas O, Q y R 
cols_importantes = df.iloc[:, [14, 16, 17]]
cols_importantes.columns = ["DescripcionSerie", "Fecha", "Valor"]

print(cols_importantes.head())
print(cols_importantes.tail())

      DescripcionSerie      Fecha Valor
0  Caudal medio diario 1950-01-01  9266
1  Caudal medio diario 1950-01-02  9130
2  Caudal medio diario 1950-01-03  8926
3  Caudal medio diario 1950-01-04  8705
4  Caudal medio diario 1950-01-05  8370
                DescripcionSerie      Fecha                Valor
27736  Temperatura máxima diaria 1992-03-09  2025-06-28 00:00:00
27737  Temperatura máxima diaria 1992-03-10  2025-01-29 00:00:00
27738  Temperatura máxima diaria 1992-03-11                 31.2
27739  Temperatura máxima diaria 1992-03-12                   28
27740  Temperatura máxima diaria 1992-03-13  2025-02-28 00:00:00


In [433]:
# Filtrar solo las filas donde la descripción sea "Caudal medio diario"
caudal_df = cols_importantes[cols_importantes["DescripcionSerie"] == "Caudal medio diario"].copy()

print(caudal_df.head())
print(caudal_df.tail())
print(f"\nFilas totales después del filtro: {len(caudal_df)}")

      DescripcionSerie      Fecha Valor
0  Caudal medio diario 1950-01-01  9266
1  Caudal medio diario 1950-01-02  9130
2  Caudal medio diario 1950-01-03  8926
3  Caudal medio diario 1950-01-04  8705
4  Caudal medio diario 1950-01-05  8370
          DescripcionSerie      Fecha            Valor
26783  Caudal medio diario 2024-02-19  304537500000001
26784  Caudal medio diario 2024-02-20  305723437499994
26785  Caudal medio diario 2024-02-21  309796875000005
26786  Caudal medio diario 2024-02-22  304692187500001
26787  Caudal medio diario 2024-02-23  302379687500003

Filas totales después del filtro: 26788


In [434]:
caudal_df["Valor"].apply(type).value_counts()

Valor
<class 'int'>                  26468
<class 'str'>                    223
<class 'datetime.datetime'>       97
Name: count, dtype: int64

Valores tomados como fechas. Hay que volverlos números

In [435]:
# Prueba unitaria
fila_prueba = caudal_df.iloc[[24063]]
valor_prueba = fila_prueba["Valor"].iloc[0]
print(valor_prueba)

5391-05-01 00:00:00


In [436]:
from datetime import datetime

# Ver cuántos valores son tipo fecha en la columna 'Valor'
mask_fechas = caudal_df["Valor"].apply(lambda x: isinstance(x, (pd.Timestamp, datetime, np.datetime64)))

print("Cantidad de valores tipo fecha en 'Valor':", mask_fechas.sum())
print("Índices (primeros 10):", caudal_df.index[mask_fechas][:10].tolist())
print("\nEjemplos de filas con fechas:")
print(caudal_df.loc[mask_fechas, ["Fecha", "Valor"]])


Cantidad de valores tipo fecha en 'Valor': 97
Índices (primeros 10): [23987, 23988, 24063, 24068, 24469, 24488, 24490, 24493, 24494, 24495]

Ejemplos de filas con fechas:
           Fecha                Valor
23987 2016-04-27  4054-01-01 00:00:00
23988 2016-04-28  4057-06-01 00:00:00
24063 2016-07-12  5391-05-01 00:00:00
24068 2016-07-17  4988-05-01 00:00:00
24469 2017-08-23  8809-06-01 00:00:00
...          ...                  ...
25589 2020-11-07  9534-09-01 00:00:00
25593 2020-11-11  9637-02-01 00:00:00
25597 2020-11-15  9798-09-01 00:00:00
25871 2021-08-16  9728-05-01 00:00:00
25872 2021-08-17  9729-06-01 00:00:00

[97 rows x 2 columns]


In [437]:
def corregir_fecha(valor):
    # Si el valor es una fecha, transformarlo a año.mes
    if isinstance(valor, (pd.Timestamp, pd.DatetimeIndex)) or hasattr(valor, "year"):
        return float(f"{valor.year}.{valor.month}")
    else:
        return valor

caudal_df["Valor"] = caudal_df["Valor"].apply(corregir_fecha)

# Verificar los cambios
print(caudal_df.loc[24061:24064, ["DescripcionSerie", "Fecha", "Valor"]])

print("Valores aún tipo fecha después de la corrección:",
      caudal_df["Valor"].apply(lambda x: isinstance(x, (pd.Timestamp, np.datetime64))).sum())

          DescripcionSerie      Fecha            Valor
24061  Caudal medio diario 2016-07-10  534306249999995
24062  Caudal medio diario 2016-07-11         53895625
24063  Caudal medio diario 2016-07-12           5391.5
24064  Caudal medio diario 2016-07-13  538278125000002
Valores aún tipo fecha después de la corrección: 0


In [438]:
caudal_df["Valor"].apply(type).value_counts()

Valor
<class 'int'>      26468
<class 'str'>        223
<class 'float'>       97
Name: count, dtype: int64

Valores tomados como str. Hay que volverlos números

In [439]:
ejemplos_str = caudal_df.loc[caudal_df["Valor"].apply(type) == str, "Valor"].unique()[:20]
print(ejemplos_str)

['2671.25' '4763.75' '10164.9' '10200.5' '10227.2' '10253.9' '10289.5'
 '10316.2' '10298.4' '10280.6' '10147.1' '10111.5' '7043.75' '10285.05'
 '10458.6' '10701.25' '10872.25' '10885.75' '10671.1' '8808.45']


In [440]:
mask_str = caudal_df["Valor"].apply(lambda x: isinstance(x, str))
print("Strings antes:", mask_str.sum())

def parse_num_str(s: str):
    s = s.strip().replace("\u00a0", "").replace("\u202f", "") 
    # Caso con punto y coma: tomar como decimal el último separador y eliminar el otro
    if "," in s and "." in s:
        last_comma = s.rfind(",")
        last_dot = s.rfind(".")
        if last_comma > last_dot:  # coma es decimal
            s = s.replace(".", "").replace(",", ".")
        else:                      # punto es decimal
            s = s.replace(",", "")
    else:
        # Solo comas - decimal con punto
        if "," in s:
            s = s.replace(",", ".")
        # Solo puntos y más de uno - quitar miles y dejar el último como decimal
        elif s.count(".") > 1:
            parts = s.split(".")
            s = "".join(parts[:-1]) + "." + parts[-1]
    try:
        return float(s)
    except ValueError:
        return np.nan

caudal_df.loc[mask_str, "Valor"] = caudal_df.loc[mask_str, "Valor"].map(parse_num_str)

# Verificación
print("Strings después:", caudal_df["Valor"].apply(lambda x: isinstance(x, str)).sum())

Strings antes: 223
Strings después: 0


In [441]:
print(caudal_df.head())
print(caudal_df.tail())

      DescripcionSerie      Fecha Valor
0  Caudal medio diario 1950-01-01  9266
1  Caudal medio diario 1950-01-02  9130
2  Caudal medio diario 1950-01-03  8926
3  Caudal medio diario 1950-01-04  8705
4  Caudal medio diario 1950-01-05  8370
          DescripcionSerie      Fecha            Valor
26783  Caudal medio diario 2024-02-19  304537500000001
26784  Caudal medio diario 2024-02-20  305723437499994
26785  Caudal medio diario 2024-02-21  309796875000005
26786  Caudal medio diario 2024-02-22  304692187500001
26787  Caudal medio diario 2024-02-23  302379687500003


In [442]:
caudal_df["Valor"].apply(type).value_counts()

Valor
<class 'int'>      26468
<class 'float'>      320
Name: count, dtype: int64

Todo listo con los datos. Seguimos al promedio

In [443]:
n_grandes = (caudal_df["Valor"] > 16000).sum()
print("Valores mayores a 16 000:", n_grandes)

Valores mayores a 16 000: 2450


Primero hasta 2015. Todos los datos están buenos

In [444]:
caudal_df["Fecha"] = pd.to_datetime(caudal_df["Fecha"], errors="coerce")
caudal_df["Año"] = caudal_df["Fecha"].dt.year
caudal_df["Mes"] = caudal_df["Fecha"].dt.month

caudal_hasta_2015 = caudal_df[caudal_df["Fecha"] <= "2015-12-31"].copy()

promedios_2015 = (
    caudal_hasta_2015
    .groupby(["Año", "Mes"])["Valor"]
    .mean()
    .reset_index(name="PromedioMensual")
    .sort_values(["Año", "Mes"])
)

print(promedios_2015)

      Año  Mes PromedioMensual
0    1950    1     6431.322581
1    1950    2     6180.392857
2    1950    3     6484.580645
3    1950    4          6031.7
4    1950    5          7553.0
..    ...  ...             ...
784  2015    8     4127.064516
785  2015    9     4394.366667
786  2015   10     4343.516129
787  2015   11     5802.566667
788  2015   12     4444.548387

[789 rows x 3 columns]


In [445]:
# Datos desde enero 2016 
caudal_desde_2016 = caudal_df[caudal_df["Fecha"] >= "2016-01-01"].copy()

promedios_2016 = (
    caudal_desde_2016[caudal_desde_2016["Valor"] <= 16000]
    .groupby(["Año", "Mes"])["Valor"]
    .mean()
    .reset_index(name="PromedioMensual")
    .sort_values(["Año", "Mes"])
)

print(promedios_2016)

     Año  Mes PromedioMensual
0   2016    2          2428.0
1   2016    3         2671.25
2   2016    4         4055.85
3   2016    5          6155.0
4   2016    6     5834.666667
..   ...  ...             ...
67  2023    6          7745.0
68  2023    7          7252.0
69  2023    9          6075.0
70  2023   10          5790.5
71  2024    2         2889.75

[72 rows x 3 columns]


In [446]:
# Faltantes
fechas_esperadas = pd.date_range(start="2016-01-01", end=caudal_desde_2016["Fecha"].max(), freq="MS")
esperado = pd.DataFrame({"Año": fechas_esperadas.year, "Mes": fechas_esperadas.month})
actual = promedios_2016[["Año", "Mes"]]

faltantes = esperado.merge(actual, on=["Año", "Mes"], how="left", indicator=True)
faltantes = faltantes[faltantes["_merge"] == "left_only"].drop(columns="_merge")

print(f"\nTotal de meses esperados desde 2016: {len(esperado)}")
print(f"Meses con promedio calculado: {len(actual)}")
print(f"Meses faltantes sin promedio: {len(faltantes)}")


Total de meses esperados desde 2016: 98
Meses con promedio calculado: 72
Meses faltantes sin promedio: 26


In [447]:
print(faltantes)

     Año  Mes
0   2016    1
9   2016   10
12  2017    1
13  2017    2
14  2017    3
28  2018    5
30  2018    7
40  2019    5
45  2019   10
47  2019   12
50  2020    3
51  2020    4
53  2020    6
54  2020    7
55  2020    8
60  2021    1
62  2021    3
64  2021    5
72  2022    1
73  2022    2
75  2022    4
85  2023    2
91  2023    8
94  2023   11
95  2023   12
96  2024    1


In [448]:
ref_mes = (
    promedios_2015.groupby("Mes")["PromedioMensual"]
    .mean()
    .to_dict()
)

rellenos = faltantes.assign(
    PromedioMensual=faltantes["Mes"].map(ref_mes)
)

# Unir promedios 
promedios_2016_completo = (
    pd.concat([promedios_2016, rellenos], ignore_index=True)
      .sort_values(["Año", "Mes"])
      .reset_index(drop=True)
)

# Verificación
print(f"Meses faltantes rellenados: {len(rellenos)}")
print(promedios_2016_completo)

Meses faltantes rellenados: 26
     Año  Mes PromedioMensual
0   2016    1     7004.967742
1   2016    2          2428.0
2   2016    3         2671.25
3   2016    4         4055.85
4   2016    5          6155.0
..   ...  ...             ...
93  2023   10          5790.5
94  2023   11     9944.151519
95  2023   12     9909.949795
96  2024    1     7004.967742
97  2024    2         2889.75

[98 rows x 3 columns]


In [449]:
# Unir ambos periodos
promedios_totales = (
    pd.concat([promedios_2015, promedios_2016_completo], ignore_index=True)
    .sort_values(["Año", "Mes"])
    .reset_index(drop=True)
)

# Verificación
print("Filas totales:", len(promedios_totales))
print("Primeros registros:")
print(promedios_totales.head(12))
print("\nÚltimos registros:")
print(promedios_totales.tail(12))


Filas totales: 887
Primeros registros:
     Año  Mes PromedioMensual
0   1950    1     6431.322581
1   1950    2     6180.392857
2   1950    3     6484.580645
3   1950    4          6031.7
4   1950    5          7553.0
5   1950    6          9652.5
6   1950    7        11480.25
7   1950    8     11014.83871
8   1950    9    10745.966667
9   1950   10     9761.741935
10  1950   11    10763.333333
11  1950   12        11606.25

Últimos registros:
      Año  Mes PromedioMensual
875  2023    3          6244.0
876  2023    4          6300.5
877  2023    5          6844.0
878  2023    6          7745.0
879  2023    7          7252.0
880  2023    8     7448.810362
881  2023    9          6075.0
882  2023   10          5790.5
883  2023   11     9944.151519
884  2023   12     9909.949795
885  2024    1     7004.967742
886  2024    2         2889.75


In [450]:
# Verificar que estén todos los meses
años = range(promedios_totales["Año"].min(), promedios_totales["Año"].max() + 1)
meses = range(1, 13)

esperado = pd.MultiIndex.from_product([años, meses], names=["Año", "Mes"]).to_frame(index=False)

# Comparar con los que existen realmente
actual = promedios_totales[["Año", "Mes"]].drop_duplicates()

# Encontrar los que faltan
faltantes_final = esperado.merge(actual, on=["Año", "Mes"], how="left", indicator=True)
faltantes_final = faltantes_final[faltantes_final["_merge"] == "left_only"].drop(columns="_merge")

total_esperado = len(esperado)
total_actual = len(actual)
total_faltante = len(faltantes_final)

print(f"Total de meses esperados: {total_esperado}")
print(f"Meses presentes: {total_actual}")
print(f"Meses faltantes: {total_faltante}")

if total_faltante > 0:
    print("\nMeses faltantes:")
    print(faltantes_final)
else:
    print("No faltan meses.")

Total de meses esperados: 900
Meses presentes: 887
Meses faltantes: 13

Meses faltantes:
      Año  Mes
83   1956   12
250  1970   11
251  1970   12
890  2024    3
891  2024    4
892  2024    5
893  2024    6
894  2024    7
895  2024    8
896  2024    9
897  2024   10
898  2024   11
899  2024   12


In [451]:
import pandas as pd

# Último mes REAL presente en promedios_totales (por ejemplo 2024-02)
ultimo = pd.to_datetime(dict(year=[promedios_totales["Año"].max()],
                             month=[promedios_totales.loc[promedios_totales["Año"].idxmax(),"Mes"]],
                             day=[1]))[0]

# Generar meses esperados exactos: desde min año-mes hasta 'ultimo'
inicio = pd.Timestamp(year=promedios_totales["Año"].min(), month=1, day=1)
fechas_esperadas = pd.date_range(start=inicio, end=ultimo, freq="MS")
esperado = pd.DataFrame({"Año": fechas_esperadas.year, "Mes": fechas_esperadas.month})

# Actual (sin duplicados)
actual = promedios_totales[["Año", "Mes"]].drop_duplicates()

# Faltantes
faltantes_final = (esperado.merge(actual, on=["Año","Mes"], how="left", indicator=True)
                          .query('_merge == "left_only"')[["Año","Mes"]])

print(f"Total de meses esperados: {len(esperado)}")      # debería dar 890
print(f"Meses presentes: {len(actual)}")                  # te está dando 889
print(f"Meses faltantes: {len(faltantes_final)}")
print(faltantes_final)

Total de meses esperados: 889
Meses presentes: 887
Meses faltantes: 3
      Año  Mes
83   1956   12
250  1970   11
251  1970   12


In [452]:
rellenos_finales = pd.DataFrame({
    "Año": [1956, 1970, 1970],
    "Mes": [12, 11, 12],
})
rellenos_finales["PromedioMensual"] = rellenos_finales["Mes"].map(ref_mes)

# Concatenar con la tabla total y ordenar
promedios_totales = (
    pd.concat([promedios_totales, rellenos_finales], ignore_index=True)
    .sort_values(["Año", "Mes"])
    .reset_index(drop=True)
)

# Verificar
print("Filas totales después del relleno:", len(promedios_totales))
print("\nÚltimos valores añadidos:")
print(rellenos_finales)

Filas totales después del relleno: 890

Últimos valores añadidos:
    Año  Mes  PromedioMensual
0  1956   12      9909.949795
1  1970   11      9944.151519
2  1970   12      9909.949795


In [453]:
print(promedios_totales)
nombre = "promedios_totales.xlsx"
promedios_totales.sort_values(["Año", "Mes"]).to_excel(nombre, index=False)
print(f"\nArchivo '{nombre}' generado exitosamente.")

      Año  Mes PromedioMensual
0    1950    1     6431.322581
1    1950    2     6180.392857
2    1950    3     6484.580645
3    1950    4          6031.7
4    1950    5          7553.0
..    ...  ...             ...
885  2023   10          5790.5
886  2023   11     9944.151519
887  2023   12     9909.949795
888  2024    1     7004.967742
889  2024    2         2889.75

[890 rows x 3 columns]

Archivo 'promedios_totales.xlsx' generado exitosamente.


Anomalía estandarizada

In [454]:
df_mensual = promedios_totales.rename(columns={"PromedioMensual": "Valor"}).copy()

# Calcular el promedio y desviación estándar para cada mes
estadisticas_mensuales = (
    df_mensual.groupby("Mes")["Valor"]
    .agg(["mean", "std"])
    .reset_index()
    .rename(columns={"mean": "Promedio mes", "std": "Desviacion mes"})
)

# Unir al DataFrame de promedios
df_anomalias = pd.merge(df_mensual, estadisticas_mensuales, on="Mes", how="left")

# Evitar divisiones por cero (si algún mes tiene desviación 0)
df_anomalias["Desviacion mes"].replace(0, np.nan, inplace=True)

# Calcular anomalía estandarizada
df_anomalias["Anomalia"] = (df_anomalias["Valor"] - df_anomalias["Promedio mes"]) / df_anomalias["Desviacion mes"]

print(df_anomalias.head())
print(df_anomalias.tail())

# Guardar el resultado
df_anomalias_final = df_anomalias[["Año", "Mes", "Valor", "Anomalia"]].sort_values(["Año", "Mes"])
df_anomalias_final.to_excel("Calamar_anomalias.xlsx", index=False)
print("\nArchivo 'Calamar_anomalias.xlsx' generado exitosamente.")

    Año  Mes        Valor Promedio mes  Desviacion mes  Anomalia
0  1950    1  6431.322581  6956.786796     2214.733338 -0.237258
1  1950    2  6180.392857  4442.228638     1527.288943  1.138072
2  1950    3  6484.580645  4200.476657     1388.711827  1.644765
3  1950    4       6031.7  5080.241404     1307.573792  0.727652
4  1950    5       7553.0  6914.934238     1489.296893  0.428434
      Año  Mes        Valor Promedio mes  Desviacion mes  Anomalia
885  2023   10       5790.5  8654.910124     1959.018929 -1.462166
886  2023   11  9944.151519  9969.712316     1937.581261 -0.013192
887  2023   12  9909.949795  9992.755776     2250.128926 -0.036801
888  2024    1  7004.967742  6956.786796     2214.733338  0.021755
889  2024    2      2889.75  4442.228638     1527.288943 -1.016493


C:\Users\USUARIO\AppData\Local\Temp\ipykernel_11988\2991971351.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_anomalias["Desviacion mes"].replace(0, np.nan, inplace=True)



Archivo 'Calamar_anomalias.xlsx' generado exitosamente.
